In [69]:
import os
import cv2
import random
import rasterio
import numpy as np
from PIL import Image, ImageEnhance
import geopandas as gpd
from rasterio.mask import mask
from rasterio.plot import show
import matplotlib.pyplot as plt
from shapely.geometry import box
from torchvision import transforms
from rasterio.windows import Window

In [75]:
def save_raster(output_raster, data, profile):
    """
    Save raster data while keeping metadata.
    """
    with rasterio.open(output_raster, "w", **profile) as dest:
        for i in range(data.shape[0]):
            dest.write(data[i], i + 1)

def rotateAndDeleteEmptySpace(fileName, outputName, degreeOfRotation):
    # Open the raster .tif file
    with rasterio.open(fileName) as src:
        # Read the data from the raster (shape will be (bands, height, width))
        data = src.read()

        # Get the metadata and affine transformation
        profile = src.profile
        transform = src.transform

        cropped_bands = []
        for i in range(data.shape[0]):  # Loop over each band
            band = data[i]  # Get the i-th band
            pil_band = Image.fromarray(band)
            rotated_band = pil_band.rotate(degreeOfRotation, expand=True)
            rotated_band_data = np.array(rotated_band)
            
            # Crop empty space
            non_empty_pixels = rotated_band_data > 0
            non_empty_rows = np.any(non_empty_pixels, axis=1)
            non_empty_cols = np.any(non_empty_pixels, axis=0)
            
            min_row, max_row = np.where(non_empty_rows)[0][[0, -1]]
            min_col, max_col = np.where(non_empty_cols)[0][[0, -1]]
            
            cropped_band_data = rotated_band_data[min_row:max_row+1, min_col:max_col+1]
            cropped_bands.append(cropped_band_data)

        # Stack the cropped bands into a multi-band image
        cropped_data = np.stack(cropped_bands)

        # Update the profile and save the multi-band image
        profile.update(width=cropped_data.shape[2], height=cropped_data.shape[1])

        save_raster(outputName, cropped_data, profile)

def flip_raster(input_raster, output_raster):
    """
    Flip raster image horizontally.
    """
    with rasterio.open(input_raster) as src:
        profile = src.profile.copy()
        data = src.read()

        # Flip all bands
        flipped_data = np.array([np.fliplr(data[i]) for i in range(data.shape[0])])
        # flipped_data = np.array([np.flipud(data[i]) for i in range(data.shape[0])])

        save_raster(output_raster, flipped_data, profile)
    
def crop_raster(input_raster, output_raster, offsetRatio):
    """
    Crop a multilayer raster file to the specified bounds.
    
    Parameters:
    - input_raster: Path to the input raster file.
    - output_raster: Path to save the cropped raster file.
    - offsetRatio: crop ratio
    """
    with rasterio.open(input_raster) as src:
        # Create a geometry for the crop bounds
        bounds = src.bounds
        xOffset = (bounds.top - bounds.bottom) * offsetRatio
        yOffset = (bounds.right - bounds.left) * offsetRatio
        crop_bounds = (bounds.left + yOffset, bounds.bottom + xOffset, 
                       bounds.right - yOffset, bounds.top - xOffset)
        crop_geom = [box(*crop_bounds)]

        # Crop the raster using the geometry
        out_image, out_transform = mask(src, crop_geom, crop=True)

        # Update metadata for the new cropped raster
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform,
            "dtype": src.dtypes[0],  # Preserve original dtype
            "nodata": src.nodata,  # Preserve nodata value
            "compress": src.profile.get("compress", "LZW"),  # Preserve compression
            "photometric": src.tags().get("photometric", "RGB"),  # Preserve color interpretation
        })

        save_raster(output_raster, out_image, out_meta)

def rescale_raster(input_raster, output_raster, scale_factor):
    """
    Rescale raster image (Zoom In & Out).
    """
    with rasterio.open(input_raster) as src:
        profile = src.profile.copy()
        data = src.read()

        new_h, new_w = int(data.shape[1] * scale_factor), int(data.shape[2] * scale_factor)

        # Resize each band
        rescaled_data = np.array([cv2.resize(data[i], (new_w, new_h), interpolation=cv2.INTER_LINEAR) for i in range(data.shape[0])])

        # Crop back to original size if zoomed in
        if scale_factor > 1:
            start_h, start_w = (new_h - data.shape[1]) // 2, (new_w - data.shape[2]) // 2
            rescaled_data = rescaled_data[:, start_h:start_h + data.shape[1], start_w:start_w + data.shape[2]]

        profile.update(height=data.shape[1], width=data.shape[2])

        save_raster(output_raster, rescaled_data, profile)

def add_gaussian_noise_raster(input_raster, output_raster, std=25):
    """
    Add Gaussian noise to raster image.
    """
    with rasterio.open(input_raster) as src:
        profile = src.profile.copy()
        data = src.read().astype(np.float32)

        # Add noise
        noise = np.random.normal(0, std, data.shape).astype(np.float32)
        noisy_data = np.clip(data + noise, 0, 255).astype(np.uint8)

        save_raster(output_raster, noisy_data, profile)

def apply_color_jitter_raster(input_raster, output_raster):
    """
    Apply color jittering (Hue, Saturation).
    """
    with rasterio.open(input_raster) as src:
        profile = src.profile.copy()
        data = src.read()

        # Convert to PIL Image
        img = np.transpose(data[:3], (1, 2, 0)).astype(np.uint8)  # Select first 3 bands (RGB)
        img = Image.fromarray(img)

        transform = transforms.ColorJitter(saturation=1.5, hue=0.2)
        img = transform(img)

        # Convert back to numpy and save
        img_array = np.array(img).transpose(2, 0, 1)  # Convert to (C, H, W)
        data[:3] = img_array  # Apply only to first 3 bands

        save_raster(output_raster, data, profile)

def apply_gamma_correction(input_raster, output_raster, gamma=1.2):
    """
    Apply gamma correction to a raster image.
    """
    with rasterio.open(input_raster) as src:
        profile = src.profile.copy()
        data = src.read().astype(np.float32)

        # Normalize data to [0,1] range
        data = data / 255.0

        # Apply gamma correction
        data = np.power(data, gamma)

        # Scale back to [0,255]
        data = np.clip(data * 255, 0, 255).astype(np.uint8)

        save_raster(output_raster, data, profile)


In [76]:
# Example Usage
input_raster_name = "normal_87_fixed"
input_raster_file = input_raster_name + ".tif"
# rotateAndDeleteEmptySpace(input_raster_file, input_raster_name + "_rotated_90.tif", 90)
# rotateAndDeleteEmptySpace(input_raster_file, input_raster_name + "_rotated_180.tif", 180)
# rotateAndDeleteEmptySpace(input_raster_file, input_raster_name + "_rotated_270.tif", 270)
# flip_raster(input_raster_file, input_raster_name + "_flipped2.tif")
# crop_raster(input_raster_file, input_raster_name + "_center_cropped.tif", 0.15)
# rescale_raster(input_raster_file, input_raster_name + "_zoomed_in.tif", 1.5)
# rescale_raster(input_raster_file, input_raster_name + "_zoomed_out.tif", 0.5)
# add_gaussian_noise_raster(input_raster_file, input_raster_name + "_noisy.tif", 10)
# add_gaussian_noise_raster(input_raster_file, input_raster_name + "_noisy.tif", 25)
# apply_color_jitter_raster(input_raster_file, input_raster_name + "_color_jittered.tif")
apply_gamma_correction(input_raster_file, input_raster_name + "_bright_contrast_lb.tif", 0.8)
apply_gamma_correction(input_raster_file, input_raster_name + "_bright_contrast_lc.tif", 1.2)